In [2]:
from qutip import * 
import numpy as np
from scipy.linalg import null_space

In [3]:
import numpy as np
from qutip import Qobj

def print_quantum_state_as_superposition(state):
    """
    Helper function to print quantum state as superposition of basis functions.
    Handles both real and imaginary amplitudes and ensures proper formatting.

    Parameters:
    state (Qobj): The quantum state to print.

    Example Output:
    |ψ⟩ = - 0.50|0000⟩ + 0.50|0011⟩ + 0.50|1100⟩ - 0.50|1111⟩
    """
    # Ensure the input is a Qobj
    if not isinstance(state, Qobj):
        raise ValueError("Input must be a Qobj.")

    # Normalize the state
    state = state.unit()

    # Determine the number of qubits
    dimension = int(np.log2(state.shape[0]))

    # Flatten the state vector
    state_vector = state.full().flatten()

    # Basis states for binary representation
    basis_states = ['0', '1']

    superposition = []
    for i, amplitude in enumerate(state_vector):
        abs_amplitude = np.abs(amplitude)

        # Ignore negligible amplitudes
        if round(abs_amplitude, 2) > 0:
            # Determine the sign
            sign = '-' if amplitude.real < 0 else '+'

            # Handle imaginary amplitudes
            if np.iscomplex(amplitude) and abs(amplitude.imag) > 1e-10:
                amplitude_str = f"{abs(amplitude):.2f}i"
            else:
                amplitude_str = f"{abs(amplitude):.2f}"

            # Construct the binary state
            binary_state = ''.join(basis_states[int(bit)] for bit in f"{i:0{dimension}b}")
            superposition.append(f"{sign} {amplitude_str}|{binary_state}⟩")

    # Remove the first sign if it is '+'
    if superposition and superposition[0][0] == '+':
        superposition[0] = superposition[0][2:]

    # Combine the terms into a single string
    superposition_str = ' '.join(superposition).replace('+-', '- ')

    # Print the result
    print(f"|ψ⟩ = {superposition_str}")

In [4]:
def find_code_space(stabilizers):
    """
    Find the code space (logical space) corresponding to a given set of stabilizers.

    Parameters:
    stabilizers (list of Qobj): List of stabilizer operators.

    Returns:
    code_states (list of Qobj): List of quantum states that form the code space.
    """
    # Initialize the full Hilbert space dimension
    dim = stabilizers[0].shape[0]

    # Start with the full Hilbert space and project it onto the +1 eigenspace of each stabilizer
    projector = Qobj(np.eye(dim))  # Start with the identity operator

    for S in stabilizers:
        # Find the +1 eigenspace of the current stabilizer
        evals, evecs = S.eigenstates()
        plus_one_vectors = [v.full() for v, e in zip(evecs, evals) if abs(e - 1) < 1e-10]

        if not plus_one_vectors:
            raise ValueError("No +1 eigenspace found for one of the stabilizers.")

        # Construct the projection operator for the +1 eigenspace
        plus_one_space = np.hstack(plus_one_vectors)
        projector_plus_one = plus_one_space @ plus_one_space.conj().T

        # Project the current projector onto the +1 eigenspace of the stabilizer
        projector = Qobj(projector_plus_one) @ projector

    # The code space is the range of the final projector
    # Find the eigenstates of the projector with eigenvalue 1
    evals, evecs = projector.eigenstates()
    code_states = [v for v, e in zip(evecs, evals) if abs(e - 1) < 1e-10]
    
    for code_state in code_states:
        # print(code_state)
        print_quantum_state_as_superposition(code_state)

In [31]:
X = sigmax()
Y = sigmay()
Z = sigmaz()
I = qeye(2)

# stabilizers = [
#     tensor(X, X, X, X),
#     tensor(Z, Z, Z, Z),
#     tensor(X, Y, Z, I)
# ]

# stabilizers = [
#     tensor(I, Z, Z),
#     tensor(Z, Z, I)
# ]
# 
# 
stabilizers = [
    tensor(X, X, X, X),
    tensor(Z, Z, Z, Z)
]
     

find_code_space(stabilizers)

|ψ⟩ = - 0.71|0011⟩ - 0.71|1100⟩
|ψ⟩ = - 0.71|0110⟩ - 0.71|1001⟩
|ψ⟩ = - 0.71|0101⟩ - 0.71|1010⟩
|ψ⟩ = 0.71|0000⟩ + 0.71|1111⟩
